In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import sklearn.tree as tree
from sklearn.svm import SVC
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.model_selection import learning_curve
from sklearn.tree import DecisionTreeClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn import svm
import decimal
import sklearn.datasets

In [2]:
def float_range(start, stop, step):
  while start < stop:
    yield float(start)
    start += decimal.Decimal(step)


In [3]:
X_bc, y_bc = sklearn.datasets.load_breast_cancer(return_X_y=True)
X_train_bc, X_test_bc, y_train_bc, y_test_bc = train_test_split(X_bc, y_bc, random_state=0, test_size=0.4)

df = pd.read_csv("occupancy_data.csv")
#df = sqlContext.read.csv("/FileStore/tables/occ.csv", header=True).toPandas()
X_occ = df.iloc[:,:5]
y_occ = df.iloc[:,5]
X_train_occ, X_test_occ, y_train_occ, y_test_occ = train_test_split(X_occ, y_occ, random_state=0, test_size=0.4)



In [4]:
dt_training_pd = {"bc_alpha":[], "bc_training":[], "bc_test":[], "occ_alpha":[], "occ_training":[], "occ_test":[]}
for ccp_alpha in list(float_range(0, 0.4, '0.005')):
    clf = DecisionTreeClassifier(random_state=0, criterion='gini', ccp_alpha=ccp_alpha, max_depth=5)
    dt_training_pd["bc_alpha"].append(ccp_alpha)
    dt_training_pd["bc_training"].append(np.mean(cross_val_score(estimator=clf, X=X_train_bc, y=y_train_bc, cv=5, n_jobs=4)))
    dt_training_pd["bc_test"].append(np.mean(cross_val_score(estimator=clf, X=X_test_bc, y=y_test_bc, cv=5, n_jobs=4)))

for ccp_alpha in list(float_range(0, 0.4, '0.005')):
    clf = DecisionTreeClassifier(random_state=0, criterion='gini', ccp_alpha=ccp_alpha, max_depth=5)
    dt_training_pd["occ_alpha"].append(ccp_alpha)
    dt_training_pd["occ_training"].append(np.mean(cross_val_score(estimator=clf, X=X_train_occ, y=y_train_occ, cv=5, n_jobs=4)))
    dt_training_pd["occ_test"].append(np.mean(cross_val_score(estimator=clf, X=X_test_occ, y=y_test_occ, cv=5, n_jobs=4)))

clf_bc = DecisionTreeClassifier(random_state=0, criterion='gini', ccp_alpha=0.025, max_depth=5)
clf_occ = DecisionTreeClassifier(random_state=0, criterion='gini', ccp_alpha=0.005, max_depth=5)

train_sizes_bc, train_scores, test_scores = learning_curve(clf_bc, X_train_bc, y_train_bc.ravel(), cv=5,n_jobs=4, train_sizes=np.linspace(.1, 1.0, 10))
train_scores_mean_bc,test_scores_mean_bc = np.mean(train_scores, axis=1),np.mean(test_scores, axis=1)

train_sizes_occ, train_scores, test_scores = learning_curve(clf_occ, X_train_occ, y_train_occ.ravel(), cv=5,n_jobs=4, train_sizes=np.linspace(.1, 1.0, 10))
train_scores_mean_occ,test_scores_mean_occ = np.mean(train_scores, axis=1),np.mean(test_scores, axis=1)

final_df = pd.DataFrame({"percentage":np.linspace(.1, 1.0, 10), "bc_training":train_scores_mean_bc, "bc_test":test_scores_mean_bc, "occ_train":train_scores_mean_occ, "occ_test":test_scores_mean_occ})
print(pd.DataFrame(dt_training_pd))
print(final_df)


bc_alpha bc_training bc_test occ_alpha occ_training occ_test
0 0.000 0.909165 0.929952 0.000 0.994063 0.986802
1 0.005 0.915047 0.929952 0.005 0.988946 0.985880
2 0.010 0.920844 0.938744 0.010 0.988946 0.985880
3 0.015 0.929582 0.956135 0.015 0.988946 0.985880
4 0.020 0.923700 0.956135 0.020 0.988946 0.985880
.. ... ... ... ... ... ...
76 0.380 0.621697 0.689372 0.380 0.792631 0.780233
77 0.385 0.621697 0.636039 0.385 0.792631 0.780233
78 0.390 0.621697 0.636039 0.390 0.792631 0.780233
79 0.395 0.621697 0.636039 0.395 0.792631 0.780233
80 0.400 0.621697 0.636039 0.400 0.792631 0.780233

[81 rows x 6 columns]
 percentage bc_training bc_test occ_train occ_test
0 0.1 1.000000 0.826598 0.997436 0.988127
1 0.2 0.977778 0.920801 0.993086 0.988946
2 0.3 0.982716 0.909207 0.994027 0.988946
3 0.4 0.957407 0.917860 0.991811 0.988946
4 0.5 0.967647 0.915047 0.992221 0.988946
5 0.6 0.953374 0.923743 0.991894 0.988946
6 0.7 0.957895 0.923743 0.991005 0.988946
7 0.8 0.947465 0.920801 0.990147 0.988946
8 0.9 0.950000 0.920759 0.989537 0.988332
9 1.0 0.949265 0.917818 0.989150 0.988946

In [5]:
nn_training_pd = {"bc_alpha":[], "bc_training":[], "bc_test":[], "occ_alpha":[], "occ_training":[], "occ_test":[]}
for i in range(1, 50, 5):
    clf = MLPClassifier(hidden_layer_sizes=i)
    nn_training_pd["bc_alpha"].append(i)
    nn_training_pd["bc_training"].append(np.mean(cross_val_score(estimator=clf, X=X_train_bc, y=y_train_bc, cv=5, n_jobs=4)))
    nn_training_pd["bc_test"].append(np.mean(cross_val_score(estimator=clf, X=X_test_bc, y=y_test_bc, cv=5, n_jobs=4)))

for i in range(1, 50, 5):
    clf = MLPClassifier(hidden_layer_sizes=i)
    nn_training_pd["occ_alpha"].append(i)
    nn_training_pd["occ_training"].append(np.mean(cross_val_score(estimator=clf, X=X_train_occ, y=y_train_occ, cv=5, n_jobs=4)))
    nn_training_pd["occ_test"].append(np.mean(cross_val_score(estimator=clf, X=X_test_occ, y=y_test_occ, cv=5, n_jobs=4)))

clf_bc = MLPClassifier(hidden_layer_sizes=10)
clf_occ = MLPClassifier(hidden_layer_sizes=5)

train_sizes_bc, train_scores, test_scores = learning_curve(clf_bc, X_train_bc, y_train_bc.ravel(), cv=5,n_jobs=4, train_sizes=np.linspace(.1, 1.0, 10))
train_scores_mean_bc,test_scores_mean_bc = np.mean(train_scores, axis=1),np.mean(test_scores, axis=1)

train_sizes_occ, train_scores, test_scores = learning_curve(clf_occ, X_train_occ, y_train_occ.ravel(), cv=5,n_jobs=4, train_sizes=np.linspace(.1, 1.0, 10))
train_scores_mean_occ,test_scores_mean_occ = np.mean(train_scores, axis=1),np.mean(test_scores, axis=1)

final_df = pd.DataFrame({"percentage":np.linspace(.1, 1.0, 10), "bc_training":train_scores_mean_bc, "bc_test":test_scores_mean_bc, "occ_train":train_scores_mean_occ, "occ_test":test_scores_mean_occ})
print(pd.DataFrame(nn_training_pd))
print(final_df)

In [6]:
boost_training_pd = {"bc_alpha":[], "bc_training":[], "bc_test":[], "occ_alpha":[], "occ_training":[], "occ_test":[]}
for i in range(1, 100, 10):
    clf = AdaBoostClassifier(n_estimators=i, learning_rate=0.9)
    boost_training_pd["bc_alpha"].append(i)
    boost_training_pd["bc_training"].append(np.mean(cross_val_score(estimator=clf, X=X_train_bc, y=y_train_bc, cv=5, n_jobs=4)))
    boost_training_pd["bc_test"].append(np.mean(cross_val_score(estimator=clf, X=X_test_bc, y=y_test_bc, cv=5, n_jobs=4)))

for i in range(1, 100, 10):
    clf = AdaBoostClassifier(n_estimators=i, learning_rate=0.9)
    boost_training_pd["occ_alpha"].append(i)
    boost_training_pd["occ_training"].append(np.mean(cross_val_score(estimator=clf, X=X_train_occ, y=y_train_occ, cv=5, n_jobs=4)))
    boost_training_pd["occ_test"].append(np.mean(cross_val_score(estimator=clf, X=X_test_occ, y=y_test_occ, cv=5, n_jobs=4)))

clf_bc = AdaBoostClassifier(n_estimators=20)
clf_occ = AdaBoostClassifier(n_estimators=20)

train_sizes_bc, train_scores, test_scores = learning_curve(clf_bc, X_train_bc, y_train_bc.ravel(), cv=5,n_jobs=4, train_sizes=np.linspace(.1, 1.0, 10))
train_scores_mean_bc,test_scores_mean_bc = np.mean(train_scores, axis=1),np.mean(test_scores, axis=1)

train_sizes_occ, train_scores, test_scores = learning_curve(clf_occ, X_train_occ, y_train_occ.ravel(), cv=5,n_jobs=4, train_sizes=np.linspace(.1, 1.0, 10))
train_scores_mean_occ,test_scores_mean_occ = np.mean(train_scores, axis=1),np.mean(test_scores, axis=1)

final_df = pd.DataFrame({"percentage":np.linspace(.1, 1.0, 10), "bc_training":train_scores_mean_bc, "bc_test":test_scores_mean_bc, "occ_train":train_scores_mean_occ, "occ_test":test_scores_mean_occ})
print(pd.DataFrame(boost_training_pd))
print(final_df)

In [7]:
svm_training_pd = {"bc_alpha":[], "bc_training":[], "bc_test":[], "occ_alpha":[], "occ_training":[], "occ_test":[]}
for i in ["linear", "poly"]:
    clf = SVC(kernel=i)
    clf.fit(X_train_bc, y_train_bc)
    svm_training_pd["bc_alpha"].append(i)
    svm_training_pd["bc_training"].append(accuracy_score(y_train_bc, clf.predict(X_train_bc)))
    clf.fit(X_test_bc, y_test_bc)
    svm_training_pd["bc_test"].append(accuracy_score(y_test_bc, clf.predict(X_test_bc)))

for i in ["linear", "poly"]:
    clf = SVC(kernel=i)
    clf.fit(X_train_occ, y_train_occ)
    svm_training_pd["occ_alpha"].append(i)
    svm_training_pd["occ_training"].append(accuracy_score(y_train_occ, clf.predict(X_train_occ)))
    clf.fit(X_test_occ, y_test_occ)
    svm_training_pd["occ_test"].append(accuracy_score(y_test_occ, clf.predict(X_test_occ)))

clf_bc = SVC(kernel='linear')
clf_occ = SVC(kernel='linear')

train_sizes_bc, train_scores, test_scores = learning_curve(clf_bc, X_train_bc, y_train_bc.ravel(), cv=5,n_jobs=4, train_sizes=np.linspace(.1, 1.0, 10))
train_scores_mean_bc,test_scores_mean_bc = np.mean(train_scores, axis=1),np.mean(test_scores, axis=1)

train_sizes_occ, train_scores, test_scores = learning_curve(clf_occ, X_train_occ, y_train_occ.ravel(), cv=5,n_jobs=4, train_sizes=np.linspace(.1, 1.0, 10))
train_scores_mean_occ,test_scores_mean_occ = np.mean(train_scores, axis=1),np.mean(test_scores, axis=1)

final_df = pd.DataFrame({"percentage":np.linspace(.1, 1.0, 10), "bc_training":train_scores_mean_bc, "bc_test":test_scores_mean_bc, "occ_train":train_scores_mean_occ, "occ_test":test_scores_mean_occ})
print(pd.DataFrame(svm_training_pd))
print(final_df)

In [8]:
knn_training_pd = {"bc_alpha":[], "bc_training":[], "bc_test":[], "occ_alpha":[], "occ_training":[], "occ_test":[]}
for i in range(1, 100, 10):
    clf = KNeighborsClassifier(n_neighbors=i)
    knn_training_pd["bc_alpha"].append(i)
    knn_training_pd["bc_training"].append(np.mean(cross_val_score(estimator=clf, X=X_train_bc, y=y_train_bc, cv=5, n_jobs=4)))
    knn_training_pd["bc_test"].append(np.mean(cross_val_score(estimator=clf, X=X_test_bc, y=y_test_bc, cv=5, n_jobs=4)))

for i in range(1, 100, 10):
    clf = KNeighborsClassifier(n_neighbors=i)
    knn_training_pd["occ_alpha"].append(i)
    knn_training_pd["occ_training"].append(np.mean(cross_val_score(estimator=clf, X=X_train_occ, y=y_train_occ, cv=5, n_jobs=4)))
    knn_training_pd["occ_test"].append(np.mean(cross_val_score(estimator=clf, X=X_test_occ, y=y_test_occ, cv=5, n_jobs=4)))

clf_bc = KNeighborsClassifier(n_neighbors=10)
clf_occ = KNeighborsClassifier(n_neighbors=11)

train_sizes_bc, train_scores, test_scores = learning_curve(clf_bc, X_train_bc, y_train_bc.ravel(), cv=5,n_jobs=4, train_sizes=np.linspace(.1, 1.0, 10))
train_scores_mean_bc,test_scores_mean_bc = np.mean(train_scores, axis=1),np.mean(test_scores, axis=1)

train_sizes_occ, train_scores, test_scores = learning_curve(clf_occ, X_train_occ, y_train_occ.ravel(), cv=5,n_jobs=4, train_sizes=np.linspace(.1, 1.0, 10))
train_scores_mean_occ,test_scores_mean_occ = np.mean(train_scores, axis=1),np.mean(test_scores, axis=1)

final_df = pd.DataFrame({"percentage":np.linspace(.1, 1.0, 10), "bc_training":train_scores_mean_bc, "bc_test":test_scores_mean_bc, "occ_train":train_scores_mean_occ, "occ_test":test_scores_mean_occ})
print(pd.DataFrame(knn_training_pd))
print(final_df)